In [1]:
from pathlib import Path
import torchaudio

## Data loading

In [2]:
for i, loc in enumerate(Path("../data/audio_segments/").glob("*")):
    print(torchaudio.info(loc))
    if i == 20:
        break

AudioMetaData(sample_rate=48000, num_frames=222408, num_channels=2, bits_per_sample=0, encoding=OPUS)
AudioMetaData(sample_rate=44100, num_frames=4166, num_channels=2, bits_per_sample=0, encoding=AAC)
AudioMetaData(sample_rate=44100, num_frames=584, num_channels=2, bits_per_sample=0, encoding=AAC)
AudioMetaData(sample_rate=48000, num_frames=697608, num_channels=2, bits_per_sample=0, encoding=OPUS)
AudioMetaData(sample_rate=44100, num_frames=10476, num_channels=1, bits_per_sample=0, encoding=AAC)
AudioMetaData(sample_rate=48000, num_frames=11782728, num_channels=1, bits_per_sample=0, encoding=OPUS)
AudioMetaData(sample_rate=48000, num_frames=269448, num_channels=2, bits_per_sample=0, encoding=OPUS)
AudioMetaData(sample_rate=48000, num_frames=479688, num_channels=2, bits_per_sample=0, encoding=OPUS)
AudioMetaData(sample_rate=48000, num_frames=263688, num_channels=2, bits_per_sample=0, encoding=OPUS)
AudioMetaData(sample_rate=48000, num_frames=530568, num_channels=2, bits_per_sample=0, en

It seems that sample rates, channels and encodings vary quite much. Maybe it would have been good to fix these in the splitting phase. The varying lengths will also be a problem, in setting up batches.